# Convolutional Neural Networks for Dogs and Cats

## Preparation (40 points total)
#### [10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).  

In [1]:
%%time
from PIL import Image
import pandas as pd
import numpy as np

image_size = (20, 20)
X = []
y = []
label = [0, 1] #cat == 0, dog == 1
for i in range(0, 12000):
    image = Image.open('train/cat.' + str(i) +'.jpg')
    image = image.resize(image_size)
    image = image.convert('L')
    image_array = np.array(image).reshape(400,)
    X.append(image_array)
    y.append(0)
    
for i in range(0, 12000):
    image = Image.open('train/dog.' + str(i) +'.jpg')
    image = image.resize(image_size)
    image = image.convert('L')
    image_array = np.array(image).reshape(400,)
    X.append(image_array)
    y.append(1)

X = np.asarray(X)
y = np.asarray(y)

Wall time: 50.3 s


In [2]:
print(X.shape)
print(y.shape)

X = X.astype(np.float32) / 255.0 # normalize

(24000, 400)
(24000,)


#### [15 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

#### [15 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice.

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

y_ohe_train = keras.utils.to_categorical(y_train, 2)


Using TensorFlow backend.


## Modeling (50 points total)
#### [20 points] Create a convolutional neural network to use on your data using tensorflow. 

In [4]:
%%time
cnn2 = Sequential()
cnn2.add(Reshape((1, 20, 20), input_shape=(1, 400)))
cnn2.add(Conv2D(filters=16, kernel_size=(2, 2), padding='same'))
cnn2.add(Activation('relu'))
cnn2.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))


cnn2.add(Flatten())
cnn2.add(Dense(2))
cnn2.add(Activation('softmax'))

cnn2.compile(loss='mean_squared_logarithmic_error',
            optimizer='adam',
            metrics=['accuracy'])

cnn2.fit(np.expand_dims(X_train, axis=1), y_ohe_train, batch_size=32, epochs=50, shuffle=True, verbose=0)

Wall time: 1min 13s


In [5]:
from sklearn import metrics as mt

yhat2 = np.argmax(cnn2.predict(np.expand_dims(X_test, axis=1)), axis=1)
print(yhat2.shape)
acc2 = mt.accuracy_score(y_test, yhat2)
acc2

(4800,)


0.6645833333333333

#### [20 points] Investigate at least two different deep network architectures. Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab. Visualize the results of the CNNs. 

In [6]:
%%time
cnn = Sequential()

cnn.add(Reshape((1, 20, 20), input_shape=(1, 400)))
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

cnn.add(Dropout(.25))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Dropout(.25))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))


cnn.add(Flatten())
cnn.add(Dense(2))
cnn.add(Activation('softmax'))

cnn.compile(loss='mean_squared_logarithmic_error',
            optimizer='adam',
            metrics=['accuracy'])

cnn.fit(np.expand_dims(X_train, axis=1), y_ohe_train, batch_size=32, epochs=50, shuffle=True, verbose=1)

Epoch 1/75
19200/19200 [==============================] - 12s - loss: 0.1212 - acc: 0.5545    
Epoch 2/75
19200/19200 [==============================] - 11s - loss: 0.1175 - acc: 0.5981    
Epoch 3/75
19200/19200 [==============================] - 10s - loss: 0.1149 - acc: 0.6120    
Epoch 4/75
19200/19200 [==============================] - 11s - loss: 0.1138 - acc: 0.6170    
Epoch 5/75
19200/19200 [==============================] - 11s - loss: 0.1120 - acc: 0.6266    
Epoch 6/75
19200/19200 [==============================] - 11s - loss: 0.1113 - acc: 0.6343    
Epoch 7/75
19200/19200 [==============================] - 11s - loss: 0.1103 - acc: 0.6390    
Epoch 8/75
19200/19200 [==============================] - 11s - loss: 0.1100 - acc: 0.6397    
Epoch 9/75
19200/19200 [==============================] - 11s - loss: 0.1090 - acc: 0.6437    
Epoch 10/75
19200/19200 [==============================] - 11s - loss: 0.1085 - acc: 0.6491    
Epoch 11/75
19200/19200 [========================

In [7]:
yhat = np.argmax(cnn.predict(np.expand_dims(X_test, axis=1)), axis=1)
acc = mt.accuracy_score(y_test, yhat)
acc

0.66645833333333337

#### [10 points] Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve.

In [10]:
mlp = Sequential()
mlp.add( Dense(input_dim=X_train.shape[1], units=30, activation='relu') )
mlp.add( Dense(units=15, activation='relu') )
mlp.add( Dense(2) )
mlp.add( Activation('softmax') )

mlp.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

mlp.fit(X_train, y_ohe_train, 
        batch_size=32, epochs=150, 
        shuffle=True, verbose=0)

In [12]:
yhat_mlp = np.argmax(mlp.predict(X_test), axis=1)
acc_mlp = mt.accuracy_score(y_test,yhat_mlp)
acc_mlp

0.60291666666666666

## Exceptional Work (10 points total)
#### One idea: Visualize the convolutional filters chosen by your CNN. Try to interpret some of their meanings.